In [ ]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.bin.gz
!gunzip cc.id.300.bin.gz

--2025-11-28 10:04:28--  https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.id.300.bin.gz
Resolving dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)... 13.225.143.54, 13.225.143.99, 13.225.143.109, ...
Connecting to dl.fbaipublicfiles.com (dl.fbaipublicfiles.com)|13.225.143.54|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4507049071 (4.2G) [application/octet-stream]
Saving to: ‘cc.id.300.bin.gz’

cc.id.300.bin.gz    100%[===================>]   4.20G   243MB/s    in 18s     

2025-11-28 10:04:46 (242 MB/s) - ‘cc.id.300.bin.gz’ saved [4507049071/4507049071]



In [4]:
from gensim.models import KeyedVectors

ft = KeyedVectors.load_facebook_model("cc.id.300.vec", binary=False)

In [80]:
import pandas as pd
import numpy as np
from iterstrat.ml_stratifiers import MultilabelStratifiedShuffleSplit
from sklearn.metrics import classification_report

df = pd.read_csv("/kaggle/input/datasethatespeechmultilabel/clean_data.csv")
df = df.dropna()
X = df["Tweet"]
y = df.drop(["Tweet", "num_labels"], axis=1)

sss = MultilabelStratifiedShuffleSplit(
    n_splits=1,
    test_size=0.2,
    random_state=42
)

for train_idx, test_idx in sss.split(X, y):
    X_train = X.iloc[train_idx]
    X_test = X.iloc[test_idx]
    y_train = y.iloc[train_idx]
    y_test = y.iloc[test_idx]

print("Proporsi label train:")
print(y_train.mean())

print("\nProporsi label test:")
print(y_test.mean())

Proporsi label train:
HS_Individual    0.271706
HS_Group         0.151181
HS_Religion      0.060358
HS_Race          0.043126
HS_Physical      0.024562
HS_Gender        0.023324
dtype: float64

Proporsi label test:
HS_Individual    0.271896
HS_Group         0.151181
HS_Religion      0.060548
HS_Race          0.043031
HS_Physical      0.024752
HS_Gender        0.023229
dtype: float64


In [69]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_words = 35000
tokenizer = Tokenizer(num_words=max_words, oov_token="<unk>")
tokenizer.fit_on_texts(X_train)

X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq  = tokenizer.texts_to_sequences(X_test)

max_len = 50
X_train_pad = pad_sequences(X_train_seq, maxlen=max_len)
X_test_pad  = pad_sequences(X_test_seq, maxlen=max_len)

num_labels = y_train.shape[1]

In [70]:
import numpy as np

embedding_dim = 300
word_index = tokenizer.word_index
embedding_matrix = np.zeros((max_words, embedding_dim))

for word, idx in word_index.items():
    if idx >= max_words:
        continue
    if word in ft:
        embedding_matrix[idx] = ft[word]
    else:
        embedding_matrix[idx] = np.zeros(embedding_dim)

In [71]:
import numpy as np
from tensorflow.keras.callbacks import Callback

def exact_match_ratio(actual, pred):
    actual = np.array(actual)
    pred   = np.array(pred)
    return np.mean(np.all(actual == pred, axis=1))

def hamming_loss_(actual, pred):
    actual = np.array(actual)
    pred   = np.array(pred)
    N, L   = actual.shape

    diff = actual != pred
    return diff.sum() / (N * L)

def micro_f1(actual, pred):
    actual = np.array(actual)
    pred   = np.array(pred)

    TP = np.sum((actual == 1) & (pred == 1))
    FP = np.sum((actual == 0) & (pred == 1))
    FN = np.sum((actual == 1) & (pred == 0))

    if TP == 0:
        return 0.0

    precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
    recall    = TP / (TP + FN) if (TP + FN) > 0 else 0.0

    if precision + recall == 0:
        return 0.0

    return 2 * precision * recall / (precision + recall)

def macro_f1(actual, pred):
    actual = np.array(actual)
    pred   = np.array(pred)

    L = actual.shape[1]
    f1_scores = []

    for j in range(L):
        y_true = actual[:, j]
        y_pred = pred[:, j]

        TP = np.sum((y_true == 1) & (y_pred == 1))
        FP = np.sum((y_true == 0) & (y_pred == 1))
        FN = np.sum((y_true == 1) & (y_pred == 0))

        precision = TP / (TP + FP) if (TP + FP) > 0 else 0.0
        recall    = TP / (TP + FN) if (TP + FN) > 0 else 0.0

        if precision + recall > 0:
            f1 = 2 * precision * recall / (precision + recall)
        else:
            f1 = 0.0

        f1_scores.append(f1)

    return np.mean(f1_scores)

class MultiLabelMetrics(Callback):
    def __init__(self, X_val, y_val):
        super().__init__()
        self.X_val = X_val
        self.y_val = y_val

    def on_epoch_end(self, epoch, logs=None):
        y_prob = self.model.predict(self.X_val, verbose=0)
        y_pred = (y_prob > 0.5).astype(int)

        emr = exact_match_ratio(self.y_val, y_pred)
        ham = hamming_loss_(self.y_val, y_pred)
        mic = micro_f1(self.y_val, y_pred)
        mac = macro_f1(self.y_val, y_pred)

        print(f" — val_exact_match: {emr:.4f} — val_hamming: {ham:.4f} — val_micro_f1: {mic:.4f} — val_macro_f1: {mac:.4f}")

        logs["val_exact_match"] = emr
        logs["val_hamming_loss"] = ham
        logs["val_micro_f1"] = mic
        logs["val_macro_f1"] = mac

In [75]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional

model = Sequential()

model.add(
    Embedding(
        input_dim=max_words,
        output_dim=embedding_dim,
        weights=[embedding_matrix],
        input_length=max_len,
        trainable=False
    )
)

model.add(Dropout(0.25))

model.add(
    Bidirectional(
        LSTM(256, dropout=0.3, recurrent_dropout=0.2)
    )
)

model.add(Dense(128, activation="relu"))
model.add(Dropout(0.3))

model.add(Dense(num_labels, activation="sigmoid"))

model.compile(
    loss="binary_crossentropy",
    optimizer="adam"
)

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [76]:
for train_idx, test_idx in sss.split(X_train_pad, y_train):
    X_train_final = X_train_pad[train_idx] 
    X_val_final   = X_train_pad[test_idx]   

    y_train_final = y_train.iloc[train_idx]
    y_val_final   = y_train.iloc[test_idx] 


metrics_callback = MultiLabelMetrics(X_val_final, y_val_final)

In [77]:
history = model.fit(
    X_train_final,
    y_train_final,
    validation_data=(X_val_final, y_val_final),
    epochs=10,
    batch_size=64,
    callbacks=[metrics_callback],
    verbose=1
)

Epoch 1/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 168ms/step - loss: 0.3588 — val_exact_match: 0.5769 — val_hamming: 0.0958 — val_micro_f1: 0.0000 — val_macro_f1: 0.0000
132/132 ━━━━━━━━━━━━━━━━━━━━ 35s 214ms/step - loss: 0.3583 - val_loss: 0.2610 - val_exact_match: 0.5769 - val_hamming_loss: 0.0958 - val_micro_f1: 0.0000e+00 - val_macro_f1: 0.0000e+00
Epoch 2/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 169ms/step - loss: 0.2567 — val_exact_match: 0.6030 — val_hamming: 0.0888 — val_micro_f1: 0.1849 — val_macro_f1: 0.1458
132/132 ━━━━━━━━━━━━━━━━━━━━ 26s 198ms/step - loss: 0.2566 - val_loss: 0.2282 - val_exact_match: 0.6030 - val_hamming_loss: 0.0888 - val_micro_f1: 0.1849 - val_macro_f1: 0.1458
Epoch 3/10
132/132 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - loss: 0.2338 — val_exact_match: 0.6092 — val_hamming: 0.0871 — val_micro_f1: 0.2448 — val_macro_f1: 0.2014
132/132 ━━━━━━━━━━━━━━━━━━━━ 26s 199ms/step - loss: 0.2338 - val_loss: 0.2138 - val_exact_match: 0.6092 - val_hamming_loss: 0.0871 - val_micro_f1: 0.

In [81]:
y_pred_probs = model.predict(X_test_pad)
y_pred = (y_pred_probs > 0.5).astype(int)

print("Exact Match Ratio:", exact_match_ratio(y_test, y_pred))
print("Hamming Loss:", hamming_loss_(y_test, y_pred))
print("Micro F1:", micro_f1(y_test, y_pred))
print("Macro F1:", macro_f1(y_test, y_pred))
print("Classification Report:",classification_report(y_test, y_pred))

83/83 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step
Exact Match Ratio: 0.6881188118811881
Hamming Loss: 0.06968773800456969
Micro F1: 0.5315699658703071
Macro F1: 0.4214444903818184
Classification Report:               precision    recall  f1-score   support

           0       0.74      0.45      0.56       714
           1       0.76      0.37      0.49       397
           2       0.79      0.41      0.54       159
           3       0.69      0.72      0.70       113
           4       0.00      0.00      0.00        65
           5       0.89      0.13      0.23        61

   micro avg       0.75      0.41      0.53      1509
   macro avg       0.65      0.35      0.42      1509
weighted avg       0.72      0.41      0.52      1509
 samples avg       0.19      0.17      0.18      1509



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
